In [1]:
import os
if not os.path.exists('hpt') and os.getcwd() == '/kaggle/working':
    !git clone https://github.com/Milpo1/hpt
    os.chdir('hpt')

Cloning into 'hpt'...
remote: Enumerating objects: 33821, done.
remote: Counting objects: 100% (5591/5591), done.
remote: Compressing objects: 100% (1599/1599), done.
remote: Total 33821 (delta 4126), reused 5456 (delta 3992), pack-reused 28230
Receiving objects: 100% (33821/33821), 687.40 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (21035/21035), done.
Updating files: 100% (88/88), done.


In [2]:
import os
from matplotlib import image as mpimg
import numpy as np
from skimage.transform import resize
data_method = 'word' # word / tile

if os.path.exists('data'):
    import shutil
    shutil.rmtree('data')

In [3]:
if data_method == 'word':
        # A program for reading and displaying handwritten words downloaded from graphic 
    # files based on descriptions from text files
    import os
    from skimage.transform import resize
    from matplotlib import image as mpimg
    def trim_image(image, trim_percentage=0.9):
        new_height = int(image.shape[0] * trim_percentage)
        new_width = int(image.shape[1] * trim_percentage)

        offset_height = (image.shape[0] - new_height) // 2
        offset_width = (image.shape[1] - new_width) // 2

        cropped_image = image[offset_height:offset_height+new_height, offset_width:offset_width+new_width]

        return cropped_image
    num_of_authors = 8
    # max_num_of_words_per_author = 400000

    def crop(n, max_n):
        return max(0,n) if n <= max_n else max_n

    def merge(l, start, end):
        sub_merged = ""
        for o in l[start:end+1]:
            sub_merged = sub_merged+o
        merged = l[:start] + [sub_merged] + l[end+1:]
        return merged
    h_l, w_l = [], []
    for author_no in range(num_of_authors):
        file_desc_name = "sources/author" + str(author_no + 1) + "/word_places.txt"
        file_desc_ptr = open(file_desc_name, 'r', errors='ignore')
        text = file_desc_ptr.read()
        lines = text.split('\n')
        number_of_lines = lines.__len__() - 1
        row_values = lines[0].split()
        number_of_values = row_values.__len__()

        num_of_words = 0
        image_file_name_prev = ""
        subimage_dir = 'data/'+'a'+str(author_no + 1)

        if not os.path.exists(subimage_dir):
            os.makedirs(subimage_dir)   

        for i in range(number_of_lines):
            row_values = lines[i].split()

            if len(row_values) > 6:
                row_values = merge(row_values,1,len(row_values)-5)
            elif len(row_values) < 6:
                continue

            if row_values[0] != '%':
                num_of_words += 1
                number_of_values = len(row_values)

                image_file_name = "sources/author" + str(author_no + 1) + "/" + row_values[0][1:-1].replace('\\','/')

                if image_file_name != image_file_name_prev:   
                    image = mpimg.imread(str(image_file_name))
                    image_file_name_prev = image_file_name
                word = row_values[1]

                if word == "<brak>":
                    continue

                row1, column1, row2, column2 = int(row_values[2]), int(row_values[3]), \
                    int(row_values[4]), int(row_values[5])

                height, width = len(image), len(image[0])
                row1, row2 =  crop(row1,height), crop(row2,height)
                column1, column2 =  crop(column1,width), crop(column2,width)

                subimage = image[min(row1,row2):max(row1,row2),
                                min(column1,column2):max(column1,column2)] 
#                 subimage = trim_image(subimage,0.95)
                h_l.append(len(subimage))
                w_l.append(len(subimage[0]))

                #subimage = resize(subimage, (65,154))
                mpimg.imsave(subimage_dir+'/'+str(num_of_words)+'.bmp',subimage)

            # if num_of_words >= max_num_of_words_per_author: break


        file_desc_ptr.close()

In [4]:
if data_method == 'tile':
    num_of_authors = 8

    hor_tile = 8
    th = 0.92
    def is_empty_subimage(subimage, threshold=th):
        gray_subimage = np.mean(subimage, axis=2)

        white_pixels = np.sum(gray_subimage > 220)
        total_pixels = gray_subimage.shape[0] * gray_subimage.shape[1]
        white_percentage = white_pixels / total_pixels

        return white_percentage > threshold

    def trim_image(image, trim_percentage=0.9):
        new_height = int(image.shape[0] * trim_percentage)
        new_width = int(image.shape[1] * trim_percentage)

        offset_height = (image.shape[0] - new_height) // 2
        offset_width = (image.shape[1] - new_width) // 2

        cropped_image = image[offset_height:offset_height+new_height, offset_width:offset_width+new_width]

        return cropped_image

    print(f'th: {th} hor tile: {hor_tile}')
    if os.path.exists('data'):
        import shutil
        shutil.rmtree('data')
    for author_no in range(num_of_authors):
        source_dir = 'sources/author'+str(author_no + 1)+'/skany'
        label_dir = 'data/'+'a'+str(author_no + 1)
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)  
        files = os.listdir(source_dir)
        image_files = [file for file in files if file.endswith('bmp')]
        subimage_no = 0
        for image_file in image_files:
            image_file_name = source_dir + "/" + image_file
            image = mpimg.imread(str(image_file_name))
            image = trim_image(image)
            image_h, image_w = len(image), len(image[0])
            ver_tile = int(hor_tile * image_h/image_w)
            tile_size = int(image_w / hor_tile)
            for yp in range(ver_tile):
                for xp in range(hor_tile):
                    subimage = image[yp*tile_size:(yp+1)*tile_size,
                                     xp*tile_size:(xp+1)*tile_size]

                    if not is_empty_subimage(subimage):
                        subimage_no+=1
                        mpimg.imsave(label_dir+'/'+str(subimage_no)+'.bmp',subimage)     
        print(f'Subimages for label {author_no}: {subimage_no}')

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import  DataLoader, random_split
class AuthorImagesDataset:
    def __init__(self, root_dir, batch_size:int,DataProcent:float,transform=None):
        self.BATCH_SIZE=batch_size
        self.root_dir = root_dir
        self.transform = transform if transform else transforms.ToTensor()
        self.dataset = datasets.ImageFolder(root=self.root_dir, transform=self.transform)

        subset_length = int(len(self.dataset) * DataProcent)
        rest_length = len(self.dataset) - subset_length
        self.subset_data, _ = random_split(self.dataset, [subset_length, rest_length])


        test_length = int(len(self.subset_data) * 0.2)
        train_length = len(self.subset_data) - test_length

        # Split the dataset
        self.train_data, self.test_data = random_split(self.subset_data, [train_length, test_length])
        
        self.into_data_loaders()

    def into_data_loaders(self):


        self.train_dataloader= DataLoader(dataset=self.train_data, # use custom created train Dataset
                                     batch_size=self.BATCH_SIZE, # how many samples per batch?
                                    num_workers=4, # how many subprocesses to use for data loading? (higher = more)
                                    # pin_memory=True,
                                     shuffle=True) # shuffle the data?

        self.test_dataloader = DataLoader(dataset=self.test_data, # use custom created test Dataset
                                    batch_size=self.BATCH_SIZE, 
                                    num_workers=4, 
                                    shuffle=False) # don't usually need to shuffle testing data
        
        
        # word,label=next(iter(self.test_dataloader))
        # print(f"shape of dataloader {word.shape} \n and label {label}")

        
    
    def __len__(self):
        return len(self.dataset) 
            


# if __name__=='__main__':
#     t=RawData()
#     t.save_words_to_file(8)
#     s=AuthorImagesDataset(r'Data/Words',transform=transforms.Compose([
#             transforms.Resize(size=(64,64)),
#             transforms.TrivialAugmentWide(num_magnitude_bins=31),
#             transforms.ToTensor()
#         ]))
    

def create_dataloaders(
        DatasetDir:str,
        transform: transforms.Compose, 
        batch_size: int,
        DataProcent:float
):
    # t=RawData()
    # t.save_words_to_file(8)
    s=AuthorImagesDataset(DatasetDir,batch_size,DataProcent,transform)
    
    return s.train_dataloader,s.test_dataloader,s.dataset.classes

In [6]:
"""
Contains functions for training and testing a PyTorch model.
"""

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    
    
    model.train()
    train_loss,train_acc=0,0

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        y_logits=model(X)

        loss=loss_fn(y_logits,y)
        train_loss+=loss.item()


        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        
        y_pred_class=y_logits.argmax(dim=1)
        train_acc+=(y_pred_class==y).sum().item()/len(y_logits)

    train_loss=train_loss/len(dataloader)
    train_acc=train_acc/len(dataloader)
    
    return train_loss,train_acc


def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    
    model.eval()
    test_loss,test_acc=0,0

    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            y_logits=model(X)
            
            loss=loss_fn(y_logits,y)
            test_loss+=loss.item()

            y_pred_class=y_logits.argmax(dim=1)
            test_acc+=(y_pred_class==y).sum().item()/len(y_logits)

    test_loss=test_loss/len(dataloader)
    test_acc=test_acc/len(dataloader)

    return test_loss,test_acc


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    
    results = {"train_loss": [],
    "train_acc": [],
    "test_loss": [],
    "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                            dataloader=train_dataloader,
                                            loss_fn=loss_fn,
                                            optimizer=optimizer,
                                            device=device)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        if train_acc >= 0.99 and test_acc >= 0.99:
            return results
    return results



def plot_loss_curves(results: Dict[str, List[float]]):
    import matplotlib.pyplot as plt
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')

    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    # plt.show()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

In [7]:
"""
Contains PyTorch model code to ResNet50-ResNet152 models.
"""

import torch
import torch.nn as nn
from torchvision import transforms

class ResNetblock(nn.Module):
    '''
        Również do opisania ładnego teraz wyjebane jajca nie mam czasu na tłumaczenie tego gówna eeeelo
    '''
    def __init__(self,in_channels,out_channels,stride=1,identity_downsample=None) -> None:
        super(ResNetblock,self).__init__()

        self.expansion=4

        self.conv1=nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,padding=1,stride=stride)
        self.bn1=nn.BatchNorm2d(out_channels)
       
        self.conv2=nn.Conv2d(in_channels=out_channels,out_channels=out_channels,kernel_size=3,padding=1,stride=1)
        self.bn2=nn.BatchNorm2d(out_channels)
       
        self.conv3=nn.Conv2d(in_channels=out_channels,out_channels=out_channels*self.expansion,kernel_size=3,padding=1,stride=1)
        self.bn3=nn.BatchNorm2d(out_channels*self.expansion)
        
        
        self.relu=nn.ReLU()

        self.identity_downsample=identity_downsample
    
    def forward(self,x):
        identity=x
        # print(f"identity shape x{identity.shape}")
        
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        # print(f"after conv1 shape {x.shape}")

        x=self.conv2(x)
        x=self.bn2(x)
        # print(f"after conv2 shape {x.shape}")

        # print(f"before conv3 shape {x.shape}")
        x=self.conv3(x)
        x=self.bn3(x)
        # print(f"after conv3 shape {x.shape}")
        
        if self.identity_downsample != None:
            identity=self.identity_downsample(identity)
            # print(f"identity shape after downsample {identity.shape}")

        x+=identity
        x=self.relu(x)
        # print(f"x shape {x.shape}")

        return x
    


class ResNet(nn.Module):
    '''
        Creates the ResNet50+ architecture

        Replicates the ResNet50 architecture from the https://github.com/Machmurka/UnderstandingDeepLearning/blob/main/Learn%20PyTorch%20for%20Deep%20Learning/ResNet18layers.ipynb
        See the original architecture here: # https://arxiv.org/pdf/1512.03385.pdf

          Args:
            block: ResNetblock class
            # do napisania teraz wyjebane jajca
    '''
    def __init__(self,block:ResNetblock,img_channels,num_classes,block_num:list) -> None:
        super(ResNet,self).__init__()
        
        self.in_channels=64
 
        self.conv1=nn.Conv2d(img_channels,64,kernel_size=7,stride=2,padding=3)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.layer2=self._make_layer(block,block_num[0],64,1)
        self.layer3=self._make_layer(block,block_num[1],128,2)
        self.layer4=self._make_layer(block,block_num[2],256,2)
        self.layer5=self._make_layer(block,block_num[3],512,2)

        self.avg=nn.AvgPool2d((1,1))
        self.fc=nn.Linear(128*128*2,num_classes)


    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)

        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        x=self.avg(x)
        x=x.reshape(x.shape[0],-1)
        x=self.fc(x)
        


        # print(f"shape after layer1 {x.shape}")
        # print("\n\n LAYER 2 \n\n")
        # x=self.layer2(x)
        # print(f"shape after layer2 {x.shape}")
        # print("\n\n LAYER 3 \n\n")  
        # x=self.layer3(x)
        # print(f"shape after layer3 {x.shape}")
        # print("\n\n LAYER 4 \n\n")  
        # x=self.layer4(x)
        # print(f"shape after layer4 {x.shape}")
        # print("\n\n LAYER 5 \n\n")  
        # x=self.layer5(x)
        # print(f"shape after layer5 {x.shape}")
        # x=self.avg(x)
        # print(f"shape after avg {x.shape}")
        # x=x.reshape(x.shape[0],-1)
        # print(f"shape after reshape {x.shape}")
        # x=self.fc(x)
        # print(f"output shape {x.shape}")
        
        return x
    
    def _make_layer(self,block:ResNetblock,num_blocks,out_channels,stride):
        identity_downsample=None
        layers=[]

        if stride!=1 or self.in_channels!=out_channels*4:
            identity_downsample=nn.Sequential(
                nn.Conv2d(self.in_channels,out_channels*4,1,stride,padding=0),
                nn.BatchNorm2d(out_channels*4)
            )
            # print("stride test")
        
        layers.append(block(self.in_channels,out_channels=out_channels,stride=stride,identity_downsample=identity_downsample))
        self.in_channels=out_channels*4

        for i in range(num_blocks-1):
            layers.append(block(self.in_channels,out_channels=out_channels))

        return nn.Sequential(*layers)

In [8]:
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
from torchvision import transforms
import torchvision.models as models
# Porcent wykorzystania całych danych

device = "cuda" if torch.cuda.is_available() else "cpu"

data_dir='data'
print(device)

DATA_PROCENT=1
NUM_EPOCHS = 40
BATCH_SIZE = 128
LEARNING_RATE = 0.0001
BLOCK_NUM=[3,4,23,3]

data_transform= transforms.Compose([transforms.Resize(size=(128,128)),
                #transforms.TrivialAugmentWide(num_magnitude_bins=31),
                transforms.ToTensor()
                ])

train_dataloader, test_dataloader, class_names = create_dataloaders(
    DatasetDir=data_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE,
    DataProcent=DATA_PROCENT
)
model = ResNet(ResNetblock,3,len(class_names),BLOCK_NUM)
model = nn.DataParallel(model)
model = model.to(device)
model.train()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

cuda


In [ ]:
print(f"len train_dataloader {len(train_dataloader)*BATCH_SIZE}")
print(f"len test_dataloader {len(test_dataloader)*BATCH_SIZE}")

results=train(model=model,
                    train_dataloader=train_dataloader,
                    test_dataloader=test_dataloader,
                    loss_fn=loss_fn,
                    optimizer=optimizer,
                    epochs=NUM_EPOCHS,
                    device=device)

# train_features_batch, train_labels_batch = next(iter(train_dataloader))

plot_loss_curves(results)

len train_dataloader 11392
len test_dataloader 2944


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1410 | train_acc: 0.6688 | test_loss: 0.3109 | test_acc: 0.8815
Epoch: 2 | train_loss: 0.4363 | train_acc: 0.8784 | test_loss: 0.2211 | test_acc: 0.9293
Epoch: 3 | train_loss: 0.2685 | train_acc: 0.9181 | test_loss: 0.2148 | test_acc: 0.9263
Epoch: 4 | train_loss: 0.1475 | train_acc: 0.9474 | test_loss: 0.1399 | test_acc: 0.9501
Epoch: 5 | train_loss: 0.0985 | train_acc: 0.9656 | test_loss: 0.1685 | test_acc: 0.9423
Epoch: 6 | train_loss: 0.0824 | train_acc: 0.9718 | test_loss: 0.1455 | test_acc: 0.9385
Epoch: 7 | train_loss: 0.0728 | train_acc: 0.9768 | test_loss: 0.2290 | test_acc: 0.9253
Epoch: 8 | train_loss: 0.1043 | train_acc: 0.9688 | test_loss: 0.7467 | test_acc: 0.7935
Epoch: 9 | train_loss: 0.4828 | train_acc: 0.9273 | test_loss: 1.7975 | test_acc: 0.7330
Epoch: 10 | train_loss: 0.2146 | train_acc: 0.9345 | test_loss: 0.1920 | test_acc: 0.9355
Epoch: 11 | train_loss: 0.1595 | train_acc: 0.9613 | test_loss: 0.1335 | test_acc: 0.9562
Epoch: 12 | train_l

In [ ]:
import random
import matplotlib.pyplot as plt
import torch

# Set the model to evaluation mode
model.eval()

# Get some images from the training set
dataiter = iter(train_dataloader)
images, labels = next(dataiter)

# Select a subset of images to display
rand = random.randint(0,32)
images_subset, labels_subset = images[rand:rand+5], labels[rand:rand+5]

# Predict
with torch.no_grad():  # We don't need to calculate gradients here, so we disable gradient computation
    #images_flattened = images_subset.view(-1, input_size)
    outputs = model(images_subset)
    _, predicted = torch.max(outputs, 1)

# Plot the images along with predicted and true labels
fig, axes = plt.subplots(1, 5, figsize=(12, 2.5))
for ax, image, pred, true in zip(axes, images_subset, predicted, labels_subset):
    image_np = image.cpu().numpy().transpose((1,2,0))
    image_np = image_np * 0.5 + 0.5
    ax.imshow(image_np)
    
    ax.set_title(f'Predicted: {pred}\nTrue: {true}')
    ax.axis('off')

plt.show()

In [ ]:
def test_cat_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device,
              num_classes: int) -> Tuple[float, torch.Tensor]:
    
    model.eval()
    test_loss = 0
    correct_predictions = torch.zeros(num_classes, dtype=torch.float32, device=device)
    total_predictions = torch.zeros(num_classes, dtype=torch.float32, device=device)

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            y_logits = model(X)
            
            loss = loss_fn(y_logits, y)
            test_loss += loss.item()
            
            y_pred_class = y_logits.argmax(dim=1)
            
            for category in range(num_classes):
                category_mask = y == category
                correct_predictions[category] += (y_pred_class[category_mask] == y[category_mask]).sum().item()
                total_predictions[category] += category_mask.sum().item()

    test_loss = test_loss / len(dataloader)
    test_acc = correct_predictions / total_predictions

    return test_loss, test_acc
test_cat_step(model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            device=device,
            num_classes=8)

In [ ]:
torch.save(model,'resnet101.pth')